# The following are plans for changes to the currnet engine pipeline

(formarly called the optmised engin pipeline)

## Augmentations

### There are several categories of augmentations that can be trialled. These are listed below. 

#### In addition to they type of augmentation, is the way in which i can be applied. Options include

- Applying each augmentation to the audiofiles in the Species folder, to increase the dataset file size. This would ensure each sample receives each augmentation type. This approach could be used in the case of overlaying background audio to the files. 

- Audio file augmentations with the Audiomentations library would likely be more appropriate to have as a probability of the effect being applied, and a range in which it is applied selected from randomly. This could be applied separately, or as a combined augmentation of many

#### Augmentations to improve generalisibility:

This includes overlaying background audio to the clips. This provides a distinctly different file to train with and helps the model learn what the features are. The audio overlay will include noises that will  be commonly heard in the forest environment in the Otways National Park. Static noise is included as there may be a background humm of the microphone and processor for which the model should be trained against. Other suggestions are also to be added. A method of applying this is to have a clean sample of a background audio file, and randomly select a segment of this to overlay for each file. This could be run for each each noise type separately, or using a combined approach, or both.

- Rain noise
- Wind noise
- Water/stream noise
- Static noise

---

### Augmentations to audio files


### Audiomentations:

Augmentating the raw audio will help to increase the training dataset by applying small but significant changes to the audiofile that result in a different Mel Spectogram. There are many to choose from, but the ones most relevant to try are included below:

- TimeStretch
- PitchShift
- TimeMask
- FrequencyMask
- SevenBandParametricEQ
- AddGausianSNR
- AddBackgroundNoise
- Reverberation
- (volume shift)
- 

The parameters for which there are applied should be selectrd for within a range that is suitable for that genus.

For example, PitchShift on a bird will likely not be suitable, as their calls have a distinctive pitch. Likewise for TimeStretch.

However for other categories such as goats, cats, boar this could be a meaningful augmentation as vocalisaitons naturally lie within a range, depending on individual animals.



### Image augmentations:


After the audio file has been converted to a Mel Spectogram, the usual computer vision-based augmentations can be applied. However, once again there is only a subset for which are applicable for Mel Spectograms. Options to try include:

- time masking with black, white, and mean colour masks
- frequency masking with black, white, and mean colour masks
- contrast adjustments
- SpecAugment involving time warping, frequency masking, and time masking.
- Random scaling of the spectrogram for amplitude variation simulation.
- Application of Gaussian blur for resilience against quality degradations.
- Horizontal and vertical flips for simulating different sound directions/sources.
- 
- 

## Changes to the dataset


### The model architecture itself may not be optimal for training on the current dataset. 

### The current limitations to the dataset include:

#### Small number of audio files for some species

Endangeres species tend to have few files available, but are priority spcecies to include, so pose a issue to be resolved. Possibile solutions include:
- selectively applying many augmentations to those files to artificially increase dataset size
- creating a separate model that is trained on under-represented species
- expanding the search for more data

#### Large number of audio files for some species

There are many species for which there is an abundance of audio availaboe, and selectively decreasing the number of files used may be an option. Appling augmentations to species that are represented around the mean/median could be an option also, to match the number of files. There are commonly large number of files for pests, domestic animals, common species of birds, and frogs.

#### Together leades to an imbalance in the model's training files

---

#### Raw full-length audio files that have non-animal vocalisations. 

The audio sourced is almost always not *clean* and without proper pre-processing and cleaning of the datasource, the model not learn correctly. The commonly found issues include:

- Vocalisations of other species of that genus (eg. several species of birds in the one file)
- Vocalisations of different genus (eg. frog, insect and bird noises in the one file)
- Human voices are often present at the beginning of files, introducing that species.
    
####  Incorrectly classified segments

The cleaning process currently is to use YamNet as event detection for classes for which that vocalisation falls under. The limitations to this approach include:

- Limited set of audio event types that YamNet can classify
- Misclassificaiton of audio events by YamNet

Together this can lead to a dataset of segments of audio for which many are not vocalisations of that species. 

Other parameters include the length of audio extracted when an event is detected.

#### Current solutions being investifgated

Fine tuning of the parameters used in YamNet is the first approach to take. 
A ML approach of cluster analysis is also being trialed to group events into clusters. The number of clusters can be tweaked.
Manual cleaning of the audio files is the immediate solution to clean the data. 
Further, combining these approaches is running clustering on a speceis, and grouping clusters for which include positive audio of that species.
    


## Changes to the model 

### The model architecture can also be investigated. This involves:

#### Using different imported (transfer-learning) models

There are many computer-vision models that are pre-trained on images. The current enging pipeline use ***EfficientNetV2*** however the are many others to trial:

- VGG16
- Inception
- Xception
- EfficientNet
- 
-
- 

#### Using different imported models trained for audio classification. 

These are models that have been trained on Mel Spectograms and may provide more relevant feature embeddings for audio-based machine learning. Model to try include:

- Bird Vocalisation Classifier
- SoundNet
- VGGish

### Modifying the layers used after the imported model.

To prevent overfitting, it may be necessary to make changes to the current architecture of the model. This could include more dropout layers, or differnt kinds of condensing the network layers and using different activation functions.


## Current Model Architecture and Augmentation

### Audio and Image Augmentation Methods:

In the process of model training and validation, data augmentation plays a pivotal role in enhancing the model's ability to generalize. Our dataset comprises both audio clips and their corresponding Mel spectrogram images. To cater to the unique characteristics of each modality, we employ distinct augmentation techniques.

#### Audio Augmentation:

- **Gaussian Noise Addition**: Intermittently, Gaussian noise is introduced to the audio samples. This simulates the real-world scenario where recordings may have background noise.
  
- **Time Stretching**: Audio samples are occasionally stretched or compressed in time. This variation simulates different speaking rates and can make the model more tolerant to variations in speech speed.

#### Image Augmentation:

- **Random Rotations**: The Mel spectrogram images undergo random rotations within a range of -2 to 2 degrees. This ensures the model is robust to minor shifts or distortions in the spectrogram.

These augmentations not only enhance the diversity of our training data but also ensure that our model remains robust against minor perturbations and variations in real-world scenarios.


### Model Architecture:

1. **Input Layer**: 
   - This layer is responsible for receiving the images and has a shape of `(MODEL_INPUT_IMAGE_HEIGHT, MODEL_INPUT_IMAGE_WIDTH, MODEL_INPUT_IMAGE_CHANNELS)`.
   
2. **EfficientNetV2 Feature Generator**:
   - Utilizes the EfficientNetV2 architecture from TensorFlow Hub for feature extraction. 
   - Specifically, we are using the `efficientnet_v2_imagenet1k_b0` variant. 
   - This layer expects images of size `260x260x3`.

3. **Flatten Layer**:
   - This layer is used to convert the 2D feature maps into 1D vectors, which can be input to the subsequent dense layers.

4. **Batch Normalization**:
   - Applied post flattening to normalize the activations, promoting faster training and convergence.

5. **Dense Layers**:
   - Two fully connected layers with multiple of the number of classes (`len(class_names) * 8` and `len(class_names) * 4`). 
   - Both layers use ReLU activation and are followed by batch normalization.

6. **Dropout Layer**:
   - Dropout of `0.50` is applied to prevent overfitting.

7. **Output Layer**:
   - The final layer is a dense layer with nodes equal to the number of classes (`len(class_names)`). 
   - It provides the class scores without activation.

By utilizing the EfficientNetV2 architecture combined with custom classification layers, we aim to achieve high accuracy on our specific classification task.


## Proposed Strategies for Model Optimization

Based on our current training diagnostics, the model exhibits signs of overfitting when trained on the available dataset. This section outlines a set of structured strategies to counteract this behavior and optimize the model's performance.

### 1. **Regularization Techniques**:
   - **L1 Regularization**: Introduce an L1 penalty on the model's weights. This encourages the model to have a sparse representation, which can be beneficial in preventing overfitting.
   - **L2 Regularization**: By adding an L2 penalty to the loss function, the model's weights are prevented from growing excessively large, a common symptom leading to overfitting.

### 2. **Dropout Rate Adjustment**:
While the current dropout rate stands at `0.50`, a meticulous analysis of validation results can guide its fine-tuning:
   - **Increase Rate**: If overfitting persists, consider increasing the dropout rate to introduce higher regularization.
   - **Decrease Rate**: In scenarios where the model underfits, reducing the dropout rate may allow the model to capture more intricate patterns.

### 3. **Model Simplification**:
   - **Architecture Refinement**: Evaluate the necessity of each layer in the model. Removing superfluous layers or neurons can lead to a simpler model less prone to overfitting.
   - **Feature Selection**: Limiting the input features to only the most informative ones can reduce the complexity of the decision boundary, potentially improving generalization.

### 4. **Hyperparameter Optimization**:
   - **Grid Search**: Implement a systematic exploration of hyperparameters by testing all possible combinations within a predefined range.
   - **Random Search**: Instead of exhaustively searching all combinations, random search samples hyperparameters from a distribution over possible parameter values, offering a more efficient search strategy.
   - **Bayesian Optimization**: This probabilistic model-based approach seeks to find the hyperparameter combination that gives the maximum value of the target function, making it a robust choice for hyperparameter tuning.

### 5. **Expanding the Dataset**:
While not a direct model adjustment, obtaining more diverse data points or leveraging external datasets can assist the model in better generalization.

In light of the aforementioned strategies, continuous monitoring, testing, and iterative refinement will be pivotal in achieving optimal model performance on unseen data.

